# One Dimensional Hard Spheres

In [17]:
import time
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
from plotly.graph_objs import Layout, Scatter, Heatmap, Histogram, Figure

In [18]:
def sample2(sampling_function, n_samples, L, sigma):
    samples1 = np.empty(n_samples)
    samples2 = np.empty(n_samples)
    
    for i in range(n_samples):
        p1, p2 = sampling_function(L, sigma)
        samples1[i] = p1
        samples2[i] = p2

    return samples1, samples2 


def sample2_naive(L, sigma):
    while True:
        p1 = np.random.uniform(0 + sigma, L - sigma)
        p2 = np.random.uniform(0 + sigma, L - sigma)

        if abs(p1 - p2) > 2*sigma:
            return p1, p2


def sample2_wrong(L, sigma):
    while True:
        p1 = np.random.uniform(0 + sigma, L - sigma)
        
        while True:
            p2 = np.random.uniform(0 + sigma, L - sigma)
            if abs(p1 - p2) > 2*sigma:
                return p1, p2


class PinSampler(object):
    def __init__(self, num_spheres, length, sigma):
        self.num_spheres = num_spheres
        self.length = length
        self.sigma = sigma

        if num_spheres * 2*sigma >= length:
            raise Exception("Invalid number of spheres!")

    def sample(self, size=1):
        s = np.empty((size, self.num_spheres))
        
        for i in range(size):
            s[i] = self._sample()

        return s


class DirectPinSampler(PinSampler):
    def _sample(self):
        s = np.empty(self.num_spheres)

        i = 0
        while True:
            p = np.random.uniform(self.sigma, self.length - self.sigma)

            if self._overlap(p, s):
                s = np.empty(self.num_spheres)
                i = 0

            else:
                s[i] = p
                i += 1

                if i >= self.num_spheres:
                    return s


    def _overlap(self, p, sample):
        return np.any(np.abs(sample - p) < 2*self.sigma)
    
class InflateDeflatePinSampler(PinSampler):
    def _sample(self):
        # Deflated sample
        deflated = sorted(np.random.uniform(
            0,
            self.length - 2*self.num_spheres*self.sigma,
            size=self.num_spheres
        ))

        # Inflate
        sample = np.empty(self.num_spheres)
        for i, x in enumerate(deflated):
            sample[i] = x + (2*i + 1)*self.sigma

        return sample

## 1 Naive direct sampling of two balls

The purpose of the exercise is samplig for the distribution:

$$
\begin{equation}

$$

In [19]:
y=sample2(sample2_naive, 10**5, 8, 0.75)
data = [
    Histogram(x=y[0], histnorm="probability", name="sphere1"),
    Histogram(x=y[1], histnorm="probability", name="sphere2")
]

lyt = Layout(title="Naive direct sampling", xaxis=dict(title="pdf sphere"),
             bargroupgap=0.1)

py.plot(Figure(data=data, layout=lyt))

'file://C:\\Users\\Diego\\Desktop\\computational-science\\h4\\temp-plot.html'

## 2 Wrong direct sampling of two balls

In [ ]:
z=sample2(sample2_wrong, 10**5, 8, 0.75)

data = [
    Histogram(x=z[0], histnorm="probability", name="sphere1"),
    Histogram(x=z[1], histnorm="probability", name="sphere2")
]

lyt = Layout(title="Wrong direct sampling", xaxis=dict(title="pdf sphere"),
             bargroupgap=0.1)

py.plot(Figure(data=data, layout=lyt))

## 3 Naive direct ampling of n balls

In [13]:
Ns = list(range(1, 15))
times = []
for n in Ns:
    sampler = DirectPinSampler(n, 10, 0.1)
    start = time.process_time()
    sampler.sample(10)
    delta = time.process_time() - start
    times.append(delta)


py.plot([Scatter(x=Ns, y=times)])

'file://C:\\Users\\Diego\\Desktop\\computational-science\\h4\\temp-plot.html'

## 4 Deflation algorithm

In [22]:
Ns = list(range(1, 15))
times_direct = []
times_infdef = []
for n in Ns:
    sampler_direct = DirectPinSampler(n, 20, 0.1)
    sampler_infdef = InflateDeflatePinSampler(n, 20, 0.1)

    start = time.process_time()
    sampler_direct.sample(20)
    delta = time.process_time() - start
    times_direct.append(delta)
    
    start = time.process_time()
    sampler_infdef.sample(20)
    delta = time.process_time() - start
    times_infdef.append(delta)


py.plot([
Scatter(x=Ns, y=times_direct, name="Direct"),
Scatter(x=Ns, y=times_infdef, name="Inflate-deflate"),
 ])

'file://C:\\Users\\Diego\\Desktop\\computational-science\\h4\\temp-plot.html'

## 5 Probability Distribution of n balls

In [23]:
sampler = InflateDeflatePinSampler(10, 20, 0.75)
s = sampler.sample(10**5).flatten()
py.plot([Histogram(x=s)])

'file://C:\\Users\\Diego\\Desktop\\computational-science\\h4\\temp-plot.html'

## 6 Analytical computation of the pdf

# ???